<a href="https://colab.research.google.com/github/X-Fan-Jack/SmartTrafficLight-ABM-RL/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mesa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.3 MB/s eta 0:00:00


In [ ]:
import random
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid

class CarAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=False, include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def step(self):
        self.move()

class TrafficLightAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.state = 'red' 
        self.green_time = 0 
        self.state = "EW" 

    def step(self):
        if self.state == 'red':
            self.state = 'green'
            self.green_time = self.random.randint(10, 30)
        elif self.state == 'green':
            self.green_time -= 1
            if self.green_time <= 0:
                self.state = 'red'

class CrossroadModel(Model):
    def __init__(self, width, height):
        self.grid = MultiGrid(width, height, True)
        self.step_counter = 0

    def step(self):
        self.step_counter += 1
        if self.step_counter % 2 == 0:
            self.generate_new_car()
        self.schedule.step()

    def generate_new_car(self):
        entry_points = [(0, self.height // 2), (self.width - 1, self.height // 2), (self.width // 2, 0), (self.width // 2, self.height - 1)]
        entry_point = self.random.choice(entry_points)
        new_car = CarAgent(self.next_id(), self)
        self.grid.place_agent(new_car, entry_point)

